In [7]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import json
import pickle
from collections import defaultdict
from PIL import Image
%matplotlib inline

In [8]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Activation, AveragePooling2D, Cropping2D, Reshape, BatchNormalization
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras.losses import mean_squared_error, binary_crossentropy
from keras.preprocessing.image import Iterator
from keras.utils.np_utils import to_categorical
from keras.layers import Reshape, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.engine.topology import Layer

import tensorflow as tf
sess = tf.Session()
K.set_session(sess)

In [9]:
y_true_np = np.ones((5, 6, 10, 2))
y_pred_np = np.ones((5, 6, 10, 1))

In [10]:
y_true = K.constant(y_true_np)
y_pred = K.constant(y_pred_np)

In [11]:
y_true_weights = y_true[..., 1:2]
y_true_mask = y_true[..., 0:1]

In [12]:
loss_per_pixel = K.binary_crossentropy(y_pred, y_true_mask)

In [13]:
loss_per_pixel * y_true_weights

<tf.Tensor 'mul:0' shape=(5, 6, 10, 1) dtype=float32>

In [14]:
def weighted_binary_crossentropy(y_true, y_pred):
    y_true_weights = y_true[..., 1:2]
    y_true_mask = y_true[..., 0:1]
    loss_per_pixel = K.binary_crossentropy(y_pred, y_true_mask)
    return loss_per_pixel * y_true_weights

In [15]:
def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_mask = y_true[..., 0:1]
    y_true_f = K.flatten(y_true_mask)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [16]:
def binary_accuracy(y_true, y_pred):
    y_true_mask = y_true[..., 0:1]
    return K.mean(K.equal(y_true_mask, K.round(y_pred)), axis=-1)

In [17]:
with sess.as_default():
    g = dice_coef(y_true, y_pred).eval()

In [28]:
y_true_np = np.ones((5, 6, 10, 1))
y_pred_np = np.ones((5, 6, 10, 1))
y_true_np[0, 1, 2, 0] = 0
y_true_np[4, 0, 1, 0] = 0

In [29]:
y_true = K.constant(y_true_np)
y_pred = K.constant(y_pred_np)

In [30]:
def class_weighted_binary_crossentropy(y_true, y_pred, weights):
    loss_per_pixel = K.binary_crossentropy(y_pred, y_true)
    weights_per_pixel = y_true * weights[1] + (1 - y_true) * weights[0]
    return loss_per_pixel * weights_per_pixel

In [31]:
weights = np.array([0.2, 0.8])

In [32]:
class_weighted_binary_crossentropy(y_true, y_pred, weights)

<tf.Tensor 'mul_10:0' shape=(5, 6, 10, 1) dtype=float32>

In [33]:
weights_per_pixel = y_true * weights[1] + (1 - y_true) * weights[0]

In [36]:
with sess.as_default():
    weights_per_pixel_e = weights_per_pixel.eval()

In [38]:
weights_per_pixel_e[4, 0, 1, 0]

0.2

In [39]:
def class_weighted_binary_accuracy(y_true, y_pred, weights):
    weight_per_pixel = y_true * weights[1] + (1 - y_true) * weights[0]
    accuracy_per_pixel = K.equal(y_true, K.round(y_pred))
    return K.mean(accuracy_per_pixel * weight_per_pixel, axis=-1)

In [40]:
weight_per_pixel = y_true * weights[1] + (1 - y_true) * weights[0]

In [41]:
weight_per_pixel

<tf.Tensor 'add_7:0' shape=(5, 6, 10, 1) dtype=float32>

In [42]:
accuracy_per_pixel = K.equal(y_true, K.round(y_pred))

In [46]:
K.cast(accuracy_per_pixel, K.floatx())

<tf.Tensor 'Cast:0' shape=(5, 6, 10, 1) dtype=float32>

In [44]:
accuracy_per_pixel * weight_per_pixel

ValueError: Tensor conversion requested dtype bool for Tensor with dtype float32: 'Tensor("add_7:0", shape=(5, 6, 10, 1), dtype=float32)'